---
title: "Cuánto te afecta la semilla al resultado final"
description: "La semilla afecta a la aleatoriedad del ajuste del modelo. Esto puede generar una incertidumbre que no siempre se tiene en cuenta."
description-meta: "La semilla afecta a la aleatoriedad del ajuste del modelo. Esto puede generar una incertidumbre que no siempre se tiene en cuenta."
author: "Leonardo Hansa"
date: "2025-04-12"
categories: [exploraciones]
execute: 
  echo: true
  eval: true
  message: false
  warning: false
freeze: true
---


## Comentarios iniciales

En [Cuartil](https://open.spotify.com/episode/0Bl4iZq1o5s2Kd5WESm2Cu?si=LgiczmLkTfOpc0vKREYI9w) mencionamos que cambiar la semilla puede cambiar la métrica de ajuste de tu modelo.

Voy a ajustar un RandomForest en algún conjunto de datos varias veces. 

El objetivo es ver que, cada vez que lo ejecuto, la predicción cambia, por lo que hay cierta incertidumbre cada vez que ajustas un modelo de estos. 


In [ ]:
#| label: libs
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Preparación de datos

Los datos no me importan especialmente. Cargo unos de **sklearn** que me ha sugerido ChatGPT. Los separo en train y test. Esta separación la dejo fija: voy a estudiar cómo afecta la aleatoriedad del modelo, no la de los datos.


In [ ]:
#| label: data
data = load_diabetes()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

## Ajuste de los modelos

Ajusto 100 modelos. Voy a guardar algunas cosas de los ajustes para luego ver cómo varían de un caso a otro. Lo que quiero estudiar es si hay mucha dispersión entre unos resultados y otros.


In [ ]:
#| label: loop-model
num_trials = 200
l_r2 = []
l_preds = []

for i in range(num_trials):
    random_state = np.random.randint(0, 10000)

    model = RandomForestRegressor(random_state=random_state, n_jobs=-1)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    l_r2.append(r2_score(y_test, y_pred))

    pred_series = pd.Series(y_pred, name=f"pred_{i}")

    l_preds.append(pred_series)

Ahora creo el data frame de predicciones:


In [ ]:
#| label: concat
df_preds = pd.concat(l_preds, axis=1)

## Visualización

### Métrica de ajuste

Aquí la distribución del R2.


In [ ]:
#| label: hist-mse
#| echo: false
plt.figure(figsize=(8, 5))
plt.hist(l_r2, bins=20, color="#800080", edgecolor="black", alpha=0.7)

plt.title(
    f"Distribución de R² en {num_trials} iteraciones", fontsize=14, fontweight="bold"
)
plt.xlabel("R²", fontsize=10)
plt.ylabel("Frecuencia", fontsize=10)

plt.grid(True, linestyle="--", alpha=0.3)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.tight_layout()
plt.show()

### Predicciones

Ahora muestro cómo varía cada predicción observación a observación. Muestro solo unas pocas porque no se ve nada si intento ver todas una a una. 

Quizá no tenga sentido ver todas una a una, sino ver la desviación en general de todas las observación con respecto a su punto medio o su media o lo que sea. Pero paso de pensar. 


In [ ]:
#| label: preds-boxplots
df_long = df_preds.reset_index().melt(
    id_vars="index", var_name="modelo", value_name="pred"
)
df_long = df_long.rename(columns={"index": "id"})

subset_ids = df_long["id"].unique()[:50]
subset = df_long[df_long["id"].isin(subset_ids)]

# Calcular mediana por id
medianas = subset.groupby("id")["pred"].median().sort_values()

# Convertir 'id' a categoría ordenada según su mediana
subset["id"] = pd.Categorical(subset["id"], categories=medianas.index, ordered=True)

plt.figure(figsize=(20, 6))
sns.boxplot(data=subset, x="id", y="pred", showfliers=False)
plt.xlabel("ID de observación (subconjunto)")
plt.ylabel("Predicción")
plt.title("Boxplot de predicciones por observación (unas pocas)")
plt.xticks(rotation=90, fontsize=5)
plt.tight_layout()
plt.show()

Hay alguna observación que sí presenta algo de variabilidad entre los modelos pero no me escandizaría por ello. 
